In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
import tensorflow as tf
import hls4ml
import time

# Load and preprocess dataset
df = pd.read_csv('Dataset.csv')
le = LabelEncoder()
df['class1'] = le.fit_transform(df['class1'])

X = df.drop(columns=['class1'])
y = df['class1']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print("✅ Preprocessing done. Shapes:")
print("Train:", X_train.shape, "Test:", X_test.shape)

# Define QKeras model
model = Sequential()
model.add(QDense(
    64,
    input_shape=(X_train.shape[1],),
    name='fc1',
    kernel_quantizer=quantized_bits(8, 0, alpha=1),
    bias_quantizer=quantized_bits(8, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001),
))
model.add(QActivation(activation=quantized_relu(6), name='relu1'))
model.add(Dropout(0.3))

model.add(QDense(
    32,
    name='fc2',
    kernel_quantizer=quantized_bits(8, 0, alpha=1),
    bias_quantizer=quantized_bits(8, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001),
))
model.add(Activation(activation='softmax', name='relu2'))
model.add(Dropout(0.3))

model.add(QDense(
    len(set(y)),
    name='output',
    kernel_quantizer=quantized_bits(8, 0, alpha=1),
    bias_quantizer=quantized_bits(8, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001),
))
model.add(Activation(activation='softmax', name='softmax'))
model.add(Dropout(0.3))

model.summary()

# Compile and train
model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=2
)

# Evaluate original model
y_keras = model.predict(X_test)
y_pred_classes = np.argmax(y_keras, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"✅ Test Accuracy: {accuracy:.4f}")

# hls4ml configuration
config = hls4ml.utils.config_from_keras_model(model, granularity='name')
config['LayerName']['softmax']['exp_table_t'] = 'ap_fixed<4,1>'
config['LayerName']['softmax']['inv_table_t'] = 'ap_fixed<4,1>'
config['Model']['Strategy'] = 'Resource'
config['LayerName']['fc1']['ReuseFactor'] = 52
config['LayerName']['fc2']['ReuseFactor'] = 64
config['LayerName']['output']['ReuseFactor'] = 32

for layer in ['fc1', 'fc2', 'output']:
    config['LayerName'][layer]['Precision']['weight'] = 'ap_fixed<4,1>'
    config['LayerName'][layer]['Precision']['bias'] = 'ap_fixed<4,1>'

hls_model = hls4ml.converters.convert_from_keras_model(
    model,
    hls_config=config,
    io_type='io_stream',
    output_dir='1/hls4ml_prj_pynq',
    backend='VivadoAccelerator',
    board='pynq-z2',
    part='xc7z020clg400-1'
)
hls_model.compile()

# Predict with hls model and hardware results
y_hls = hls_model.predict(X_test)
y_hw = np.load('y_hw.npy')

y_pred_hls_classes = np.argmax(y_hls, axis=1)
y_pred_hls4ml_classes = np.argmax(y_hw, axis=1)

accuracy_hls = accuracy_score(y_test, y_pred_hls_classes)
accuracy_hls4ml = accuracy_score(y_test, y_pred_hls4ml_classes)

print(f"✅ Test Accuracy of hls : {accuracy_hls:.4f}")
print(f"✅ Test Accuracy of hls : {accuracy_hls4ml:.4f}")

np.save('x_test.npy', X_train)
np.save('y_test.npy', y_test)

print(X_train.shape)
print(y_train.shape)

# HLS Build
start = time.time()
hls_model.build(
    csim=False,
    export=True,
    bitfile=True
)
end = time.time()
print(end - start)

hls4ml.report.read_vivado_report('hls_project/hls4ml_prj_pynq')
